In [2]:
import os
import time
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import cupy as cp

input_folder = '/content/drive/MyDrive/Images/PADC_dataset_image'
output_folder = '/content/drive/MyDrive/Images/PADC_dataset_image/outputparallel'
os.makedirs(output_folder, exist_ok=True)

# Load watermark image
watermark_img_path = '/content/ich6.jpeg' # Corrected file path
watermark = Image.open(watermark_img_path).convert("RGBA").resize((50, 50))

def process_image(in_path, out_path):
    img = Image.open(in_path).resize((128, 128)).convert("RGBA")
    # Example: Convert image to numpy, then cupy for GPU operations (here, just a dummy op)
    img_np = np.array(img)
    img_cp = cp.array(img_np)
    # If you want to do some GPU-based manipulation, do it here. (We're just copying as-is.)
    img_np = cp.asnumpy(img_cp)
    img_gpu = Image.fromarray(img_np).convert("RGBA")
    # Overlay watermark
    img_gpu.paste(watermark, (10, 10), watermark)
    img_gpu = img_gpu.convert("RGB")
    img_gpu.save(out_path)

def process_folder(class_subfolder):
    class_path = os.path.join(input_folder, class_subfolder)
    out_class_path = os.path.join(output_folder, class_subfolder)
    os.makedirs(out_class_path, exist_ok=True)
    tasks = []
    for img_file in os.listdir(class_path):
        if img_file.lower().endswith(('jpg', 'jpeg', 'png')):
            in_path = os.path.join(class_path, img_file)
            out_path = os.path.join(out_class_path, img_file)
            tasks.append((in_path, out_path))
    return tasks

def parallel_process(num_workers):
    tasks = []
    class_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]
    for class_folder in class_folders:
        tasks.extend(process_folder(class_folder))
    start = time.perf_counter()
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(process_image, in_path, out_path) for in_path, out_path in tasks]
        [f.result() for f in futures]
    duration = time.perf_counter() - start
    return duration

worker_counts = [2, 4, 8]
results = []
for workers in worker_counts:
    t = parallel_process(workers)
    print(f"Workers: {workers} | Time: {t:.2f} s")
    results.append((workers, t))

# Show result table
import pandas as pd
df = pd.DataFrame(results, columns=['Workers', 'Time (s)'])
df['Speedup'] = df['Time (s)'].iloc[0] / df['Time (s)']
print(df)

Workers: 2 | Time: 0.00 s
Workers: 4 | Time: 0.00 s
Workers: 8 | Time: 0.00 s
   Workers  Time (s)   Speedup
0        2  0.000085  1.000000
1        4  0.000150  0.567628
2        8  0.000056  1.511485
